In [ ]:
import pandas as pd
from backend.src.utils.logging_handler import setup_logging
from backend.src.utils.config import Config

config = Config("../config.yaml")
setup_logging()

feature_engineering_config = config.get("feature_engineering")
preprocessor_config = config.get("preprocess")
model_config = config.get("model")

target_feature = preprocessor_config.get("target")
trend_length = preprocessor_config.get("trend_length")
window_size = preprocessor_config.get("window_size")
epochs: int = model_config.get("epochs")
batch_size: int = model_config.get("batch_size")

interval: str = feature_engineering_config.get("time_series_interval")

training_file_path: str = f"../data/twelvedata/time series ({interval})/AAPL_time_series.csv"
validation_file_path: str = f"../data/twelvedata/time series ({interval})/ABBV_time_series.csv"

scaler_path: str = "../src/scaling/saved scalers/scaler.save"

training_df = pd.read_csv(training_file_path)
validation_df = pd.read_csv(validation_file_path)


In [ ]:
from backend.src.pipelines.stock_trend_lstm_train_pipeline import StockTrendLSTMTrainPipeline
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

training_pipeline = StockTrendLSTMTrainPipeline(
    training_time_series=training_df,
    validation_time_series=validation_df,
    target_feature=target_feature,
    trend_length=trend_length,
    window_size=window_size,
    epochs=epochs,
    batch_size=batch_size,
    scaler_save_path=scaler_path,
)
training_pipeline.prepare_data()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
training_pipeline.labeled_training_ts.head(10)


In [ ]:
training_pipeline.build_model()

In [ ]:
%matplotlib inline
history = training_pipeline.train()